In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)
from sklearn.model_selection import StratifiedKFold, cross_val_predict
import random

In [2]:
seq = "agattgcaggcataagaaccaagcacagaccttccggcgacaataagcgataggattttgactggtagacagccgtactgtctttgtcaacacgtcaccttatcccaagtacttcagtagtttcaacgggactgcgtaaccgaccgtacactgactggtgaagacactattaggtcattgcgaacaacggaagatgcccacacatcaaaaggtttttttggactttagaatatccaaggtttattccccgaagatcgttctctgatgtgcctttaagagctgtactgctcggtatggaggagaaggaagtgtctattaactccctttgcccacgcatgataatgtgagtacagtacacccgaacggcctaagtcctcacattctctatcccgttactaaggcaaacgaagtctattcgggagcacaacaaccagctaacgctgaagcgaacccctacaaacgcaccgggctcttgatacggcggatggttcgccatggttgcggggattcaacagggacactatatggt".upper()
k = 6 # in (3,10)

In [3]:
def count_kmeters(seq, k):
    kmeters = {}
    def construct(seq, n):
        if n == 0:
            kmeters[seq] = 0
            return
        for c in ['A', 'C', 'G', 'T']:
            construct(seq + c, n - 1)
    construct("", k)
        
    for i in range(len(seq) - k):
        kmeters[seq[i:i+k]] += 1

    for k, v in kmeters.items():
        kmeters[k] = v / (len(k))
        
    def get_complement(c):
        if c == 'A':
            return 'T'
        if c == 'T':
            return 'A'
        if c == 'C':
            return 'G'
        if c == 'G':
            return 'C'
        return 'N'

    def get_complement_str(s):
        return "".join([get_complement(c) for c in s][::-1])

    to_delete = set()
    for k, v in kmeters.items():
        if k in to_delete:
            continue
        c = get_complement_str(k)
        if c in kmeters and c != k:
            kmeters[k] += kmeters[c]
            to_delete.add(c)
            
    for k in to_delete:
        del kmeters[k]

    return kmeters

In [4]:
kmeters = count_kmeters(seq, k)
len(kmeters)

2080

In [5]:
df = pd.read_csv("data/experiments.tsv", sep="\t")
data = df[(df["curation_status"] == "positive") | (df["curation_status"] == "negative")]
data = data.sample(frac=1).reset_index(drop=True)
data = data.loc[data['seq'].apply(lambda x: isinstance(x, str))]
data["seq"] = data["seq"].apply(lambda x: x.upper())
data = data.loc[data['seq'].apply(lambda x: set(x).issubset(set("ATCG")))]
pos_data = data[data["curation_status"] == "positive"]
neg_data = data[data["curation_status"] == "negative"]
print(len(df), len(data), len(pos_data), len(neg_data))

4646 4302 2300 2002


Negative data V2

In [6]:
length = int(np.mean([len(x) for x in data["seq"]]))
num_samples = len(neg_data)
length

2245

In [7]:
def sample_from_fasta(file_path, num_samples, length):
    sampled_sequences = []
    current_sequence = ""
    
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('>'):
                current_sequence = ""
                continue
            
            line = line.strip()
            if set(line) - {'A', 'C', 'T', 'G'}:
                current_sequence = ""
                continue
            
            current_sequence += line
            while len(current_sequence) >= length and len(sampled_sequences) < num_samples:
                start = random.randint(0, len(current_sequence) - length)
                sampled_sequences.append(current_sequence[start:start + length])
                current_sequence = current_sequence[start + length:]
            
            if len(sampled_sequences) >= num_samples:
                break

    return sampled_sequences

In [8]:
sampled_sequences = sample_from_fasta("data/GRCh38.p14.genome.fa", num_samples, length)

In [9]:
with open("data/negative.txt", "w") as file:
    file.write("\n".join(sampled_sequences))

In [10]:
print(len(sampled_sequences), len(neg_data))

2002 2002


In [11]:
X = np.concatenate([
    pos_data["seq"].values,
    neg_data["seq"].values
])
X = [count_kmeters(x, k) for x in X]
X = [list(x.values()) for x in X]
y = [1] * len(pos_data) + [0] * len(neg_data)

In [12]:
X2 = np.concatenate([
    pos_data["seq"].values,
    sampled_sequences
])
X2 = [count_kmeters(x, k) for x in X2]
X2 = [list(x.values()) for x in X2]
y2 = [1] * len(pos_data) + [0] * len(neg_data)

# Model

In [13]:
def score(X, y):
    svc = SVC(probability=True)
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    y_pred_prob = cross_val_predict(svc, X, y, cv=cv, method='predict_proba')[:, 1]
    y_pred = cross_val_predict(svc, X, y, cv=cv)

    auc = roc_auc_score(y, y_pred_prob)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)

    print(f"AUC-ROC: {auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [14]:
score(X, y)

AUC-ROC: 0.6752
Accuracy: 0.6288
Precision: 0.6351
Recall: 0.7183
F1-Score: 0.6741
Confusion Matrix:
[[1053  949]
 [ 648 1652]]


In [15]:
score(X2, y2)

AUC-ROC: 0.9896
Accuracy: 0.9519
Precision: 0.9584
Recall: 0.9513
F1-Score: 0.9548
Confusion Matrix:
[[1907   95]
 [ 112 2188]]


In [16]:
print(len(X), len(y))
print(len(X2), len(y2))

4302 4302
4302 4302
